### This Notebook contains the functions from Flodén. To use any of them, just copy the function you need into your own notebook

#### Translated from Flodén's MATLAB codes

In [1]:
# needed for both addacooper and tauchen
import numpy as np
from scipy.stats import norm

# needed for compact printing of numpy arrays
np.set_printoptions(suppress=True,precision=4,linewidth=np.inf) # use precision to set the number of decimal digits to display

# needed for addacooper only
from scipy.integrate import quad 

In [2]:
# generates an AR(1) process using Adda and Cooper's (Dynamic Economics, 2003) method
def addacooper(n,mu,rho,sigma):
    """ Syntax: [Z,PI] = addacooper(n,mu,rho,sigma)

    Approximate n-state AR(1) process following Tauchen (1986) and Tauchen & Hussey (1991).
    See Adda & Cooper (2003) pp 57-.
    Z(t+1) = mu*(1-rho) + rho*Z(t) + eps(t+1)
    where std(eps) = sigma
    Translated from Martin Flodén, 2005
    """

    sigmaUNC = sigma/np.sqrt(1-rho**2)
    E  = np.zeros((n+1,1))
    Z  = np.zeros((n,1))
    PI = np.zeros((n,n))
    MFPI = np.zeros((n,n))
    
    E[0] = -1e1
    E[-1] = 1e1
    for i in np.arange(1,n):
        E[i] = sigmaUNC*norm.ppf((i)/n) + mu
    
    #print(E)
    for i in np.arange(0,n):
        Z[i] = n*sigmaUNC*(norm.pdf((E[i]-mu)/sigmaUNC) - norm.pdf((E[i+1]-mu)/sigmaUNC)) + mu
    

    for i in np.arange(0,n):
        for j in np.arange(0,n):
            E1 = E[j]
            E2 = E[j+1]
            th_fcn = lambda u: n/np.sqrt(2*np.pi*sigmaUNC**2) * (np.exp(-(u-mu)**2 / (2*sigmaUNC**2)) * \
                      (norm.cdf((E2-mu*(1-rho)-rho*u)/sigma) - norm.cdf((E1-mu*(1-rho)-rho*u)/sigma)))
            
            PI[i,j] = quad(th_fcn,E[i],E[i+1],epsabs=1e-10)[0]
            MFPI[i,j] = norm.cdf((E[j+1]-mu*(1-rho)-rho*Z[i])/sigma) - norm.cdf((E[j]-mu*(1-rho)-rho*Z[i])/sigma)       


    for i in np.arange(0,n):
        PI[i,:] = PI[i,:] / np.sum(PI[i,:])
        MFPI[i,:] = MFPI[i,:] / np.sum(MFPI[i,:])

    return Z, PI
    

In [3]:
N = 15
rho = 0.9500
sigma = np.sqrt(0.030)
Z,PI = addacooper(N,0,rho,sigma)
print("Z =\n", Z)
print("PI =\n", PI)

Z =
 [[-1.0759]
 [-0.7153]
 [-0.5382]
 [-0.4046]
 [-0.2913]
 [-0.1892]
 [-0.0932]
 [ 0.    ]
 [ 0.0932]
 [ 0.1892]
 [ 0.2913]
 [ 0.4046]
 [ 0.5382]
 [ 0.7153]
 [ 1.0759]]
PI =
 [[0.7566 0.2015 0.0355 0.0055 0.0008 0.0001 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.2015 0.4335 0.2424 0.0889 0.0259 0.0063 0.0013 0.0002 0.     0.     0.     0.     0.     0.     0.    ]
 [0.0355 0.2424 0.3206 0.2256 0.1129 0.0443 0.0141 0.0036 0.0007 0.0001 0.     0.     0.     0.     0.    ]
 [0.0055 0.0889 0.2256 0.2671 0.2081 0.1214 0.0558 0.0204 0.0058 0.0012 0.0002 0.     0.     0.     0.    ]
 [0.0008 0.0259 0.1129 0.2081 0.2376 0.1957 0.1242 0.062  0.0241 0.0071 0.0015 0.0002 0.     0.     0.    ]
 [0.0001 0.0063 0.0443 0.1214 0.1957 0.2207 0.1881 0.1249 0.0647 0.0254 0.0071 0.0012 0.0001 0.     0.    ]
 [0.     0.0013 0.0141 0.0558 0.1242 0.1881 0.2118 0.1844 0.125  0.0647 0.0241 0.0058 0.0007 0.     0.    ]
 [0.     0.0002 0.0036 0.0204 0.062  0.1249 0.1844 0.209  0.1844 0.

In [4]:
# generates an AR(1) process using Tauchen's (Ec. Letters, 1986) method
def tauchen(N,mu,rho,sigma,m):
    """ Syntax: [Z,Zprob] = tauchen(N,mu,rho,sigma,m)
    
    Function TAUCHEN
    Purpose:    Finds a Markov chain whose sample paths
                approximate those of the AR(1) process
                    z(t+1) = (1-rho)*mu + rho * z(t) + eps(t+1)
                where eps are normal with stddev sigma
                
    Format:     {Z, Zprob} = Tauchen(N,mu,rho,sigma,m)
    
    Input:      N       scalar, number of nodes for Z
                mu      scalar, unconditional mean of process
                rho     scalar
                sigma   scalar, std. dev. of epsilons
                m       max +- std. devs.

    Output:     Z       N*1 vector, nodes for Z
                Zprob   N*N matrix, transition probabilities

    This procedure is an implementation of George Tauchen's algorithm
    described in Ec. Letters 20 (1986) 177-181.

    Translated from Martin Flodén, Fall 1996
    """

    
    
    
    Z     = np.zeros((N,1))
    Zprob = np.zeros((N,N))
    a     = (1-rho)*mu

    Z[-1]  = m * np.sqrt(sigma**2 / (1 - rho**2))
    Z[0]  = -Z[-1]
    zstep = (Z[-1] - Z[0]) / (N - 1)
    
    
    for i in np.arange(1,N-1):
        Z[i] = Z[0] + zstep * i


    Z = Z + a / (1-rho)
    

    for j in np.arange(0,N):
        for k in np.arange(0,N):
            if k == 0:
                Zprob[j,k] = norm.cdf((Z[0] - a - rho * Z[j] + zstep / 2) / sigma)
            elif k == N-1:
                Zprob[j,k] = 1 - norm.cdf((Z[-1] - a - rho * Z[j] - zstep / 2) / sigma)
            else:
                Zprob[j,k] = norm.cdf((Z[k] - a - rho * Z[j] + zstep / 2) / sigma) - \
                         norm.cdf((Z[k] - a - rho * Z[j] - zstep / 2) / sigma)
    
    return Z, Zprob

In [5]:
N = 15
rho = 0.9500
sigma = np.sqrt(0.030)
Z,PI = tauchen(N,0,rho,sigma,1.2*np.log(N))
print("Z =\n", Z)
print("PI =\n", PI)


Z =
 [[-1.8026]
 [-1.5451]
 [-1.2876]
 [-1.03  ]
 [-0.7725]
 [-0.515 ]
 [-0.2575]
 [ 0.    ]
 [ 0.2575]
 [ 0.515 ]
 [ 0.7725]
 [ 1.03  ]
 [ 1.2876]
 [ 1.5451]
 [ 1.8026]]
PI =
 [[0.5882 0.3681 0.043  0.0007 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.1171 0.4998 0.3459 0.0367 0.0005 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.0046 0.1278 0.5125 0.3235 0.0311 0.0004 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.0057 0.1433 0.5232 0.3012 0.0263 0.0003 0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.007  0.1598 0.5317 0.279  0.0221 0.0002 0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.0001 0.0086 0.1775 0.5378 0.2573 0.0185 0.0002 0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.0001 0.0105 0.1962 0.5415 0.2362 0.0154 0.0001 0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.0001 0.0128 0.2158 0.5427 0.2158 0.